<a href="https://colab.research.google.com/github/raketic-ognjen/Market-Watch-Tech-Analysis/blob/main/TechAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = "DeepSkyBlue"><center>**Market Watch - Technical Analysis**</center></font>

### <center>**Core Idea Explanation**</center>

### Adding Packages - Libraries:
1. Pandas(pandas data reader)
2. yfinance
3. NumPy(linAlg)
4. SciPy(optimize and stats)
5. Plotly(graph objects and subplots)
6. Statsmodels.api
7. DateTime

In [1]:
from google.colab import files
uploaded0=files.upload()

Saving MCF_INV.py to MCF_INV.py


In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import numpy.linalg as LA
import scipy as sp
import scipy.optimize as sco
import scipy.stats as st
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
import pandas_datareader as pdr
import datetime as dt
import MCF_INV as inv

> First of all, I want to appologise to my team and to You, professor, because I did not show up on the Market Watch Game due to my **TCL knee injury.** Because of that I will do next things:
1.   **Portfolio Optimization**
2.   **Efficient Frontiers**
3.   **CAPM Model**
4.   **Sharpe's Ratio**
5.   **Testing Portfolio Performances**
6.   **Robustness portfolio**
7.   **Black-Litterman model**
8.   **Comparing with Non-Technical Result of my group**



### <center>*Work*</center>

According to my group starting portfolio, we use 6 companies:
1. Starbucks - SBUX - 9 stocks 0.13 weight
2. Coca Cola - KO - 15  stocks 0.22 weight
3. Microsoft - MSFT 3 stocks 0.04 weight
4. Pfizer - PFE 25 stocks 0.37 weight
5. Nike - NKE 8 stocks 0.12 weight
6. Exxon Mobil - XOM 8 stocks 0.12 weight

Later Non Technical portfolio  had 83 trades and a lot buying-selling companies, so for this analysis I will use only this 6 stocks

According to TA starting portfolio, we use 6 companies:
1. Astra Zeneca - AZN -  
2. Nvidia - NVD -
3. Novartis - NVS
4. Rivian - RIVN
5. Carnival Corporation - CCL
6. Norwegian Cruise Company -NCLH

## NTA Portfolio

In [3]:
#Non Technical Analysis Portfolio start
ntaportf= ['SBUX', 'KO', 'MSFT','PFE', 'NKE', 'XOM']

### Preparation data for optimization of NTA portfolio

In [4]:
ntaprices = yf.download(ntaportf,start='2022-05-18',end='2023-05-18')['Adj Close']
#ntaprices.to_csv('ntaprices.csv',encoding='utf-8-sig')
#files.download('ntaprices.csv')

[*********************100%%**********************]  6 of 6 completed


In [7]:
rets1=pd.read_csv('ntaprices.csv',index_col=0)

In [8]:
#returns on NTA portfolio
rets1=ntaprices.pct_change().dropna()

#Weights for NTA
w1 = [0.13,0.22,0.04,0.37,0.12,0.12]

#Portfolio returns
rp1=rets1@w1
rp1

Date
2022-05-19   -0.000856
2022-05-20    0.018884
2022-05-23    0.016880
2022-05-24    0.004252
2022-05-25    0.006996
                ...   
2023-05-11   -0.007343
2023-05-12   -0.002677
2023-05-15   -0.001188
2023-05-16   -0.006898
2023-05-17    0.002667
Length: 250, dtype: float64

### From portfolio returns (Portfolio as an asset)

In [9]:
inv.expRet(rets1,True),inv.annualize_vol(rets1,True)

(KO      0.000296
 MSFT    0.001101
 NKE     0.000597
 PFE    -0.001012
 SBUX    0.001834
 XOM     0.000927
 dtype: float64,
 KO      0.010386
 MSFT    0.020836
 NKE     0.023806
 PFE     0.014571
 SBUX    0.018550
 XOM     0.020540
 dtype: float64)

In [10]:
er1= inv.expRet(rp1)
print(er1)
vol1= inv.annualize_vol(rp1)
print(vol1)

0.06586668177302485
0.19451632553064452


### From stocks' returns (Portfolio as collection of assets)

In [11]:
per1 = inv.per(W=w1,ret=rets1, n=250)
per1

0.0653439303303818

Computation risk of Portfolio

In [12]:
pV1=inv.pV(w1,rets1, 250, True)
pV1

0.19374289771402367

### MVP

In [13]:
mvp1= inv.mvp(ret=rets1,f=250,com=True)
mvp1

{'w': array([ 0.74264404, -0.01524826, -0.08163827,  0.15621441,  0.07963324,
         0.11839484]),
 'er': 0.062020684375590426,
 'vol': 0.153661656030564}

In [14]:
mvp1['w']

array([ 0.74264404, -0.01524826, -0.08163827,  0.15621441,  0.07963324,
        0.11839484])

### Manual optimization

In [15]:
res1=sco.minimize(lambda w1: inv.pV(w1,rets1),
                  [0.13,0.22,0.04,0.37,0.12,0.12],
                  constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1)])
res1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.023800703881497392
       x: [ 7.433e-01 -1.615e-02 -8.157e-02  1.548e-01  8.104e-02
            1.185e-01]
     nit: 15
     jac: [ 4.757e-02  4.712e-02  4.748e-02  4.758e-02  4.759e-02
            4.769e-02]
    nfev: 106
    njev: 15

In [16]:
target1= inv.targetP(rets1,0.1)
target1

{'w': array([ 0.75048364, -0.00959382, -0.09213339,  0.1012759 ,  0.12249179,
         0.12747588]),
 'er': 0.1,
 'vol': 0.15487292875584452}

In [17]:
result1 = sco.minimize(lambda w1: inv.pV(w1,rets1),
                  [0.13,0.22,0.04,0.37,0.12,0.12],
                  constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1),
                               dict(type='eq',fun=lambda w1: inv.per(w1,rets1)-0.08)])
result1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.023837376575403017
       x: [ 7.449e-01 -1.489e-02 -8.362e-02  1.311e-01  9.893e-02
            1.236e-01]
     nit: 11
     jac: [ 4.764e-02  4.821e-02  4.875e-02  4.618e-02  4.970e-02
            4.866e-02]
    nfev: 78
    njev: 11

In [18]:
resnta = sco.minimize(lambda w1: inv.pV(w1,rets1),
                  [1/6]*6,
                      bounds=[[0,0.04]]*6,
                  constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1),
                               dict(type='eq',fun=lambda w1: inv.per(w1,rets1)-0.1)])
resnta

 message: Iteration limit reached
 success: False
  status: 9
     fun: 0.0024526420409738472
       x: [ 4.000e-02  4.000e-02  4.000e-02  4.000e-02  4.000e-02
            4.000e-02]
     nit: 100
     jac: [ 1.083e-02  2.525e-02  3.086e-02  1.440e-02  2.246e-02
            1.884e-02]
    nfev: 1484
    njev: 100

### Efficient Frontier for NTA portfolio

In [19]:
ntaEF = inv.EF(rets1,plot='full')
ntaEF

In [20]:
fignta=inv.EF(rets1,Range=[0.01,1.5],plot='full')
fignta

In [21]:
rfa1 = pdr.get_data_fred('DTB3',start='2022-05-18',end='2023-05-18')/100
rfa1.head()
rfa1.dropna()

,DTB3
DATE,
2022-05-18,0.0102
2022-05-19,0.0104
2022-05-20,0.0101
2022-05-23,0.0107
2022-05-24,0.0105
...,...
2023-05-12,0.0508
2023-05-15,0.0506
2023-05-16,0.0507


### Deannualization(EAR)

In [22]:
rf1 = (1+rfa1)**(1/251)-1
rf1.head()
rf1.dropna()

,DTB3
DATE,
2022-05-18,0.000040
2022-05-19,0.000041
2022-05-20,0.000040
2022-05-23,0.000042
2022-05-24,0.000042
...,...
2023-05-12,0.000197
2023-05-15,0.000197
2023-05-16,0.000197


In [23]:
opt1 = inv.maxSharpe(rets1,rf1,f=251,com=True)
opt1

{'w': array([ 0.88610688,  0.27905367, -0.46499761, -1.        ,  1.        ,
         0.29983706]),
 'er': 0.9080784473783409,
 'vol': 0.34115618284742016,
 'sharpe': 2.5565580288680105}

In [24]:
fignta.add_trace(go.Scatter(x=[inv.pV(opt1['w'],rets1,n=250,vol=True)],
                            y=[inv.per(opt1['w'],rets1)],
                            marker=dict(
                                color='Crimson',
                                size=10,
                                line=dict(
                                    color='Black',
                                    width=2
                                )
                            )
                          )
)

#plot show
fignta.show()

Test for Eff frontier

In [25]:
#Initial Wealth
Wealth1 = 30_000

#Determined amount invested in each stock
si1=opt1['w']*Wealth1

#Number of stocks
n1=si1/ntaprices.iloc[0]
n1

KO      454.205155
MSFT     33.337640
NKE    -129.850332
PFE    -624.034755
SBUX    432.262344
XOM     103.501633
Name: 2022-05-18 00:00:00, dtype: float64

In [26]:
#Value of portfolio each point in the time
ST1 = ntaprices@n1
ST1

Date
2022-05-18    30000.000000
2022-05-19    29994.884107
2022-05-20    29805.069354
2022-05-23    30777.893907
2022-05-24    30859.253081
                  ...     
2023-05-11    56080.178097
2023-05-12    56765.038907
2023-05-15    57115.663545
2023-05-16    56547.501360
2023-05-17    57122.374802
Length: 251, dtype: float64

### Static Trading Strategy for NTA portfolio

In [27]:
#preparing data
return1 = ntaprices.iloc[:,:6].pct_change()
return1.dropna(inplace=True)

#initial wealth
v=30_000

#Optimal weights
ntaMVP1 = inv.mvp(return1,f=250)['w']
nta15= inv.targetP(return1,0.15)['w']

In [28]:
#optimal weights for the MVP NTA Portfolio
ntaMVP1

array([ 0.74264404, -0.01524826, -0.08163827,  0.15621441,  0.07963324,
        0.11839484])

In [29]:
#Optimal weights for portfolio with the target expected return of 15 percent per annum
nta15

array([ 0.76247647,  0.01081989, -0.11340902,  0.02548823,  0.17586989,
        0.13875455])

In [30]:
#Vector of money invested into each asset
vntaMVP1 = ntaMVP1*v
vnta15= nta15*v

In [31]:
vntaMVP1

array([22279.32122421,  -457.44781737, -2449.14796258,  4686.43218783,
        2388.99720494,  3551.84516297])

In [32]:
vnta15

array([22874.29409045,   324.59659392, -3402.27071   ,   764.64677384,
        5276.09672104,  4162.63653075])

In [33]:
#vector of innitial stock prices
ntap0 = ntaprices.iloc[0,:6]
ntap0

KO       58.526871
MSFT    251.115860
NKE     107.430824
PFE      48.074245
SBUX     69.402298
XOM      86.907921
Name: 2022-05-18 00:00:00, dtype: float64

In [34]:
#Number of shares
nMVPnta1= vntaMVP1/ntap0
n15nta1 = vnta15/ntap0

In [35]:
#number of stocks for MVP NTA is
nMVPnta1

KO      380.668246
MSFT     -1.821660
NKE     -22.797442
PFE      97.483219
SBUX     34.422451
XOM      40.869062
Name: 2022-05-18 00:00:00, dtype: float64

In [36]:
#number of stocks for portfolio with target expected return of 15% annually
n15nta1

KO      390.834053
MSFT      1.292617
NKE     -31.669409
PFE      15.905539
SBUX     76.021931
XOM      47.897090
Name: 2022-05-18 00:00:00, dtype: float64

In [37]:
#the number of shares that we need to hold over time, we can calculate the value of our portfolio on each date in testing sample simply by multiplying previous two
VntaMVP = ntaprices.iloc[:,:6]@nMVPnta1
V15nta = ntaprices.iloc[:,:6]@n15nta1

### Graphical Comparation of Static Strategy for NTA portfolio

In [38]:
fignta1 = go.Figure()
fignta1.add_trace(
    go.Scatter(x=ntaprices.index,
               y=VntaMVP,
               line=dict(color='Blue',width=3),
               name='MVP NTA portfolio')
)
fignta1.add_trace(
    go.Scatter(
        x=ntaprices.index,
        y=V15nta,
        line=dict(color='Crimson',width=3),
        name='Target NTA 15% pa')
)
fignta1.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing 2 Static Strategy for NTA portfolio',
               x = 0.5,
               y = 0.87,
               font=dict(size=25,color='Navy'))
)

fignta1.show()

In [39]:
V15nta[-1]-VntaMVP[-1]

2716.0110106978937

15% NTA Target porfolio outperform MVP NTA portfolio for 9.05%

In [40]:
ntaprices.iloc[-1,:6]

KO       62.179375
MSFT    313.336609
NKE     116.206444
PFE      36.345070
SBUX    105.129402
XOM     103.967728
Name: 2023-05-17 00:00:00, dtype: float64

In [41]:
#Weights on the last day of the trade
nMVPnta1*ntaprices.iloc[-1,:6]/VntaMVP[-1]

KO      0.742914
MSFT   -0.017915
NKE    -0.083150
PFE     0.111204
SBUX    0.113583
XOM     0.133364
dtype: float64

In [42]:
#initial Weights
ntaMVP1

array([ 0.74264404, -0.01524826, -0.08163827,  0.15621441,  0.07963324,
        0.11839484])

### Dynamic Strategy for NTA portfolio

In [99]:
#initial wealth
v1=30_000

#Optimal weights
ntaMVP = inv.mvp(return1,f=250)['w']
nta_15= inv.targetP(return1,0.15)['w']

In [100]:
ret1 = ntaprices.iloc[:,:6].pct_change()

In [102]:
ret1.dropna()

,KO,MSFT,NKE,PFE,SBUX,XOM
Date,,,,,,
2022-05-19,-0.019608,-0.003700,-0.026790,0.004960,0.009119,0.005405
2022-05-20,0.016333,-0.002291,0.014656,0.035933,0.020298,0.007900
2022-05-23,0.030830,0.032032,0.005833,0.007814,0.000409,0.022099
2022-05-24,0.018454,-0.003952,-0.012335,0.010023,-0.009534,0.005432
2022-05-25,0.000781,0.011170,0.008482,0.005617,-0.003300,0.020127
...,...,...,...,...,...,...
2023-05-11,0.005669,-0.007044,-0.010445,-0.008181,-0.007610,-0.018099
2023-05-12,0.003915,-0.003676,-0.016446,-0.006120,0.004639,-0.000095
2023-05-15,-0.002652,0.001586,-0.003161,-0.005087,0.004900,0.001907


In [104]:
# Ex-post reteurns on MVP portfolio if we keep the weights fixed
retntaMVP= (ret1@ntaMVP)
# Ex-post reteurns on nta15 portfolio if we keep the weights fixed
ret15P = (ret1@nta15)

In [106]:
retntaMVP[0]=0
ret15P[0]=0

In [109]:
VntaMVP1 = (1+retntaMVP).cumprod()*v1
V15P1= (1+ret15P).cumprod()*v1

### Graphical Comparation of Dynamic Strategy for NTA portfolio

In [113]:
figntad = go.Figure()
figntad.add_trace(go.Scatter(
    x=ntaprices.index,
    y=VntaMVP1,
    line=dict(color='Blue', width=3),
    name='MVP')
)
figntad.add_trace(go.Scatter(
    x=ntaprices.index,
    y=V15P1,
    line=dict(color='Crimson', width=3),
    name='Target 15%')
)
figntad.update_layout(
    xaxis=dict(title_text='Date', zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor= 'Black'),
    title=dict(text = 'Comparison of two dynamic strategies with constant weights',
               x=0.5,
               y=0.87,
               font=dict(size=25,
                         color='Navy'))
)

In [114]:
V15P1[-1]-VntaMVP1[-1]

2811.083619522822

# TA Portfolio

### Data Preparation

In [43]:
#Technical Analysis portfolio start
taportf = ['AZN', 'NVDA', 'NVS', 'RIVN', 'CCL', 'NCLH']

In [44]:
taprices = yf.download(taportf,start='2022-05-18',end='2023-05-18')['Adj Close']
taprices.to_csv('taprices.csv',encoding='utf-8-sig')
files.download('taprices.csv')

[*********************100%%**********************]  6 of 6 completed


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
rets2= pd.read_csv('taprices.csv',index_col=0)

### From portfolio returns (Portfolio as an asset)

In [46]:
#returns
rets2= taprices.pct_change().dropna()
rets2

#Weights
w2=[0.13,0.22,0.04,0.37,0.12,0.12]

#return on portfolio
rp2 = rets2@w2
rp2

Date
2022-05-19    0.014090
2022-05-20   -0.016601
2022-05-23    0.001928
2022-05-24   -0.047916
2022-05-25    0.030667
                ...   
2023-05-11   -0.013151
2023-05-12   -0.021508
2023-05-15    0.025521
2023-05-16    0.000646
2023-05-17    0.029414
Length: 250, dtype: float64

In [47]:
inv.expRet(rets2,250,True),inv.annualize_vol(rets2,250)

(AZN     0.205298
 CCL    -0.202312
 NCLH   -0.097484
 NVDA    0.783340
 NVS     0.204596
 RIVN   -0.508611
 dtype: float64,
 AZN     0.235327
 CCL     0.737278
 NCLH    0.705900
 NVDA    0.555384
 NVS     0.196501
 RIVN    0.731755
 dtype: float64)

In [48]:
er2 = inv.expRet(rp2)
print(er2)
vol2= inv.annualize_vol(rp2)
print(vol2)

0.2892443423842738
0.42503384133109545


### From stocks' returns (Portfolio as collection of assets)

In [49]:
#Expected Return on portfolio
per2=inv.per(w2,rets2,250,True)
per2

0.2316345512466877

In [50]:
pV2 = inv.pV(w2,rets2,250,True)
pV2

0.4233438392451838

###MVP

In [51]:
mvp2 = inv.mvp(rets2,250,True)
mvp2

{'w': array([ 0.24931181, -0.05201156,  0.05462121,  0.0201238 ,  0.69997356,
         0.02798118]),
 'er': 0.20112524194616613,
 'vol': 0.18616520269822015}

### Manual optimization

In [52]:
res2 = sco.minimize(lambda w2: pV2,
                        [0.13,0.22,0.04,0.37,0.12,0.12],
                       constraints=[dict(type='eq',fun=lambda w2:sum(w2)-1)])
res2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.4233438392451838
       x: [ 1.300e-01  2.200e-01  4.000e-02  3.700e-01  1.200e-01
            1.200e-01]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
    nfev: 7
    njev: 1

Expected return is 42.34%

In [53]:
target2=inv.targetP(rets2,0.45)
target2

{'w': array([ 0.19373691, -0.10761345,  0.06282567,  0.27633798,  0.69960892,
        -0.12489603]),
 'er': 0.45,
 'vol': 0.22170056106538172}

In [54]:
result2 = sco.minimize(lambda w2: pV2,
                       [0.13,0.22,0.04,0.37,0.12,0.12],
                       constraints=[dict(type='eq',fun=lambda w2: sum(w2)-1),
                                    dict(type='eq',fun=lambda w2: inv.per(w2,rets2,250,True)-0.45)])
result2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.4233438392451838
       x: [ 1.614e-01  1.608e-01  4.063e-03  5.299e-01  1.512e-01
           -7.353e-03]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
    nfev: 8
    njev: 1

In [55]:
tares2 = sco.minimize(lambda w2: pV2,
                       [0.13,0.22,0.04,0.37,0.12,0.12],
                        bounds=[[0,0.39]]*6,
                       constraints=[dict(type='eq',fun=lambda w2: sum(w2)-1),
                                    dict(type='eq',fun=lambda w2: inv.per(w2,rets2)-0.4)])
tares2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.4233438392451838
       x: [ 2.491e-01  8.294e-02  5.301e-02  3.900e-01  2.250e-01
            0.000e+00]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
    nfev: 8
    njev: 1

In [56]:
tares2.x

array([0.24907098, 0.0829444 , 0.05300784, 0.39      , 0.22497678,
       0.        ])

### Efficient Frontier for TA poftolio

In [57]:
taEF = inv.EF(rets2,plot='full')
taEF

In [58]:
figta=inv.EF(rets2,Range=[0.01,1.5],plot='full')
figta

In [59]:
rfa2 = pdr.get_data_fred('DTB3',start='2022-05-18',end='2023-05-18')/100
rfa2.head()
rfa2.dropna()

,DTB3
DATE,
2022-05-18,0.0102
2022-05-19,0.0104
2022-05-20,0.0101
2022-05-23,0.0107
2022-05-24,0.0105
...,...
2023-05-12,0.0508
2023-05-15,0.0506
2023-05-16,0.0507


In [60]:
rf2 = (1+rfa2)**(1/251)-1
rf2.head()
rf2.dropna()

,DTB3
DATE,
2022-05-18,0.000040
2022-05-19,0.000041
2022-05-20,0.000040
2022-05-23,0.000042
2022-05-24,0.000042
...,...
2023-05-12,0.000197
2023-05-15,0.000197
2023-05-16,0.000197


In [61]:
opt2 = inv.maxSharpe(rets2,rf2,f=251,com=True)
opt2

{'w': array([ 0.04405227, -0.3106303 ,  0.00485061,  1.        ,  0.74439785,
        -0.48267043]),
 'er': 1.2582816188188868,
 'vol': 0.48165385037101177,
 'sharpe': 2.5378988434442924}

In [62]:
figta.add_trace(go.Scatter(x=[inv.pV(opt2['w'],rets2,n=250,vol=True)],
                            y=[inv.per(opt2['w'],rets2)],
                            marker=dict(
                                color='Crimson',
                                size=10,
                                line=dict(
                                    color='Black',
                                    width=2
                                )
                            )
                          )
)

#plot show
figta.show()

### Test for Eff Frontier in TA Portfolio

In [63]:
#Initial Wealth
Wealth2 = 30_000

#Determined amount invested in each stock
si2=opt2['w']*Wealth2

#Number of stocks
n2=si2/taprices.iloc[0]
n2

AZN      21.463765
CCL    -673.331567
NCLH      9.152087
NVDA    177.315026
NVS     265.767280
RIVN   -530.601409
Name: 2022-05-18 00:00:00, dtype: float64

In [64]:
#Value of portfolio each point in the time
ST2 = taprices@n2
ST2

Date
2022-05-18    30000.000000
2022-05-19    29546.488253
2022-05-20    29679.026914
2022-05-23    30878.174027
2022-05-24    31543.272593
                  ...     
2023-05-11    65629.426322
2023-05-12    65898.758372
2023-05-15    66385.613014
2023-05-16    66925.586524
2023-05-17    67576.423016
Length: 251, dtype: float64

### Static Strategy for TA portfolio

In [74]:
#preparing data
return2 = taprices.iloc[:,:6].pct_change()
return2.dropna(inplace=True)

#initial wealth
v2=30_000

#Optimal weights
taMVP2 = inv.mvp(return2,f=250)['w']
ta30= inv.targetP(return2,0.30)['w']

In [76]:
#optimal weights for the MVP TA Portfolio
taMVP2

array([ 0.24931181, -0.05201156,  0.05462121,  0.0201238 ,  0.69997356,
        0.02798118])

In [77]:
#Optimal weights for portfolio with the target expected return of 15 percent per annum
ta30

array([ 0.22737669, -0.07334667,  0.0571907 ,  0.11895517,  0.70015859,
       -0.03033448])

In [80]:
#Vector of money invested into each asset
vtaMVP2 = taMVP2*v2
vta30= ta30*v2

In [79]:
vtaMVP2

array([ 7479.35422932, -1560.34671774,  1638.63635631,   603.71392131,
       20999.20680749,   839.43540331])

In [81]:
vta30

array([ 6821.30063773, -2200.39998607,  1715.72096274,  3568.65502347,
       21004.7577296 ,  -910.03436747])

In [82]:
#vector of innitial stock prices
tap0 = taprices.iloc[0,:6]
tap0

AZN      61.572060
CCL      13.840000
NCLH     15.900000
NVDA    169.190399
NVS      84.028160
RIVN     27.290001
Name: 2022-05-18 00:00:00, dtype: float64

In [83]:
#Number of shares
nMVPta2= vtaMVP2/tap0
n30ta2 = vta30/tap0

In [84]:
#number of stocks for MVP NTA is
nMVPta2

AZN     121.473186
CCL    -112.741814
NCLH    103.058893
NVDA      3.568252
NVS     249.906779
RIVN     30.759816
Name: 2022-05-18 00:00:00, dtype: float64

In [86]:
#number of stocks for portfolio with target expected return of 30% annually
n30ta2

AZN     110.785650
CCL    -158.988437
NCLH    107.906981
NVDA     21.092539
NVS     249.972839
RIVN    -33.346806
Name: 2022-05-18 00:00:00, dtype: float64

In [87]:
#the number of shares that we need to hold over time, we can calculate the value of our portfolio on each date in testing sample simply by multiplying previous two
VtaMVP = taprices.iloc[:,:6]@nMVPta2
V30ta = taprices.iloc[:,:6]@n30ta2

### Graphical Comparation of Static Strategy for TA portfolio

In [92]:
figta2 = go.Figure()
figta2.add_trace(
    go.Scatter(x=taprices.index,
               y=VtaMVP,
               line=dict(color='Blue',width=3),
               name='MVP TA portfolio')
)
figta2.add_trace(
    go.Scatter(
        x=taprices.index,
        y=V30ta,
        line=dict(color='Crimson',width=3),
        name='Target TA 30% pa')
)
figta2.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing 2 Static Strategy for TA portfolio',
               x = 0.5,
               y = 0.87,
               font=dict(size=25,color='Navy'))
)

figta2.show()

In [90]:
V30ta[-1]-VtaMVP[-1]

3200.3703284872463

Target TA portfolio with 10.66% outperform MVP TA portfolio

In [93]:
taprices.iloc[-1,:6]

AZN      74.212685
CCL      11.040000
NCLH     14.350000
NVDA    301.723938
NVS     101.220001
RIVN     13.410000
Name: 2023-05-17 00:00:00, dtype: float64

In [94]:
#Weights on the last day of the trade
nMVPta2*taprices.iloc[-1,:6]/VtaMVP[-1]

AZN     0.250178
CCL    -0.034542
NCLH    0.041042
NVDA    0.029878
NVS     0.701996
RIVN    0.011447
dtype: float64

In [95]:
#initial Weights
taMVP2

array([ 0.24931181, -0.05201156,  0.05462121,  0.0201238 ,  0.69997356,
        0.02798118])

# Graphs

## Graphical Comparation of Effective Frontier

In [65]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=ST1.index,y= ST1, name = 'NTA Portfolio'))
fig.add_trace(go.Scatter(x=ST2.index,y=ST2, name = 'TA Portfolio'))


fig.show()

## Graphical Comparison of Static Strategy

In [98]:
figtot = go.Figure()
figtot.add_trace(
    go.Scatter(x=ntaprices.index,
               y=VntaMVP,
               line=dict(color='Blue',width=3),
               name='MVP NTA portfolio')
)
figtot.add_trace(
    go.Scatter(
        x=ntaprices.index,
        y=V15nta,
        line=dict(color='Navy',width=3),
        name='Target NTA 15% pa')
)
figtot.add_trace(
    go.Scatter(x=taprices.index,
               y=VtaMVP,
               line=dict(color='Red',width=3),
               name='MVP TA portfolio')
)
figtot.add_trace(
    go.Scatter(
        x=taprices.index,
        y=V30ta,
        line=dict(color='Crimson',width=3),
        name='Target TA 30% pa')
)
figtot.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing NTA and TA Static Strategy',
               x = 0.5,
               y = 0.87,
               font=dict(size=25,color='Navy'))
)

figtot.show()